# Gallery Example: M/M/1 Queue with Feedback

This example demonstrates an M/M/1 queue with probabilistic feedback:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Feedback**: Jobs may return to the queue with probability p
- **Scheduling**: FCFS (First-Come-First-Served)

With feedback probability p, the effective service rate becomes μ(1-p), increasing the utilization and response time.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_feedback(p=1/3): Network {    """Create M/M/1 queueing model with feedback        Parameters:    - p: Feedback probability (default 1/3)    """    model = Network("M/M/1-Feedback")        # Block 1: nodes    source = Source(model, "Source")    queue = Queue(model, "Queue", SchedStrategy.FCFS)    sink = Sink(model, "Sink")        # Block 2: classes    oclass1 = OpenClass(model, "Class1")    source.setArrival(oclass1, Exp.fitMean(1))  # λ = 1    queue.setService(oclass1, Exp.fitMean(0.5)) # μ = 2, mean service time = 0.5        # Block 3: topology - feedback routing    P = model.initRoutingMatrix()    P.addRoute(oclass1, source, queue, 1.0)  # All arrivals go to queue    P.addRoute(oclass1, queue, queue, p)     # Feedback with probability p    P.addRoute(oclass1, queue, sink, 1-p)    # Exit with probability 1-p    model.link(P)        return model# Create the model with default feedback probabilityp_feedback = 1/3model = gallery_mm1_feedback(p_feedback)println(f"Feedback probability: {p_feedback:.3f}")

## Theoretical Analysis for M/M/1 with Feedback

For M/M/1 with feedback probability p:
- **External arrival rate**: λ = 1
- **Service rate**: μ = 2
- **Effective service rate**: μ_eff = μ(1-p) = 2(1-1/3) = 4/3
- **Effective utilization**: ρ_eff = λ/μ_eff = 1/(4/3) = 0.75
- **Mean number of visits**: 1/(1-p) = 1/(1-1/3) = 1.5
- **Throughput**: X = λ = 1.0

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 20)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare with different feedback probabilities
println("\n=== Impact of Feedback Probability ===")
val p_values = [0.0, 0.2, 0.4, 0.6, 0.8]

for p in p_values:
    val model_p = gallery_mm1_feedback(p)
    val solver = MVA(model_p)
    val avg_table = solver.avgTable
// Extract utilization and response time from results
    val util = float(avg_table.iloc[1, 1])  # Queue utilization
    val resp_time = float(avg_table.iloc[1, 2])  # Queue response time
    
    println(f"p={p:.1f}: Utilization={util:.3f}, Response Time={resp_time:.3f}")